**Predicting Investor-Owned Electric Utilities in the U.S.**

CST383 Data Science

Professor Ergezer

Jesus Martinez Miranda,

Jorge Vazquez,

Walid Elgammal,

Zuhra Totakhail

6/9/2025


Data Exploration

In [25]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import graphviz
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score



# allow output to span multiple output lines in the console
pd.set_option('display.max_columns', 500)

# switch to seaborn default stylistic parameters
# see the useful https://seaborn.pydata.org/tutorial/aesthetics.html
sns.set()
sns.set_context('paper') # 'talk' for slightly larger

# change default plot size
rcParams['figure.figsize'] = 9,7


In [ ]:
# Define packages you want to install
packages = ['numpy', 'pandas', 'matplotlib', 'seaborn']

# Loop through and install if missing
for package in packages:
    try:
        __import__(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully.")


numpy is already installed.
pandas is already installed.
matplotlib is already installed.
seaborn is already installed.


# Reading data and merging data sets

In [24]:
url1 = "https://raw.githubusercontent.com/jorgevaz47/CST383_DS_Project/working_walid/data/iou_zipcodes_2020.csv"
url2 = "https://raw.githubusercontent.com/jorgevaz47/CST383_DS_Project/working_walid/data/non_iou_zipcodes_2020.csv"

df1 = pd.read_csv(url1)
df2 = pd.read_csv(url2)

df = pd.concat([df1, df2], ignore_index=True)
print(df.shape)
df.info()
df.describe()

(80204, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80204 entries, 0 to 80203
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   zip           80204 non-null  int64  
 1   eiaid         80204 non-null  int64  
 2   utility_name  80204 non-null  object 
 3   state         80204 non-null  object 
 4   service_type  80204 non-null  object 
 5   ownership     80204 non-null  object 
 6   comm_rate     80204 non-null  float64
 7   ind_rate      80204 non-null  float64
 8   res_rate      80204 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 5.5+ MB


,zip,eiaid,comm_rate,ind_rate,res_rate
count,80204.000000,80204.000000,80204.000000,80204.000000,80204.000000
mean,46426.440863,13788.970912,0.098005,0.066949,0.115826
std,27920.227188,10680.719250,0.044314,0.044657,0.050052
min,501.000000,55.000000,0.000000,0.000000,0.000000
25%,21657.000000,6422.000000,0.078465,0.045736,0.097577
50%,44398.000000,13573.000000,0.098762,0.064780,0.115913
75%,67954.000000,17066.000000,0.118236,0.082462,0.135057
max,99950.000000,57483.000000,0.700000,0.650000,0.500285


<table border="1" style="border-collapse: collapse; width: 100%;">
  <thead>
    <tr>
      <th style="padding: 8px; text-align: left;">Column Name</th>
      <th style="padding: 8px; text-align: left;">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="padding: 8px;">zip</td>
      <td style="padding: 8px;">5-digit ZIP code representing the geographic area of the utility service</td>
    </tr>
    <tr>
      <td style="padding: 8px;">eiaid</td>
      <td style="padding: 8px;">Unique identifier for the utility company from the Energy Information Administration (EIA)</td>
    </tr>
    <tr>
      <td style="padding: 8px;">utility_name</td>
      <td style="padding: 8px;">Name of the electric utility company serving the ZIP code</td>
    </tr>
    <tr>
      <td style="padding: 8px;">state</td>
      <td style="padding: 8px;">U.S. state abbreviation where the ZIP code and utility are located</td>
    </tr>
    <tr>
      <td style="padding: 8px;">service_type</td>
      <td style="padding: 8px;">Type of customer service: residential, commercial, or industrial</td>
    </tr>
    <tr>
      <td style="padding: 8px;">ownership</td>
      <td style="padding: 8px;">Ownership category of the utility, e.g., municipal, investor-owned, cooperative</td>
    </tr>
    <tr>
      <td style="padding: 8px;">comm_rate</td>
      <td style="padding: 8px;">Average commercial electricity rate charged by the utility (cents per kWh)</td>
    </tr>
    <tr>
      <td style="padding: 8px;">ind_rate</td>
      <td style="padding: 8px;">Average industrial electricity rate charged by the utility (cents per kWh)</td>
    </tr>
    <tr>
      <td style="padding: 8px;">res_rate</td>
      <td style="padding: 8px;">Average residential electricity rate charged by the utility (cents per kWh)</td>
    </tr>
  </tbody>
</table>


# Missing Data
We noticed that no data is missing from the sets.

In [29]:
print(df.isnull().sum())


zip             0
eiaid           0
utility_name    0
state           0
service_type    0
ownership       0
comm_rate       0
ind_rate        0
res_rate        0
dtype: int64
